In [35]:
import xarray as xr
import random
import glob
import numpy as np
import matplotlib.pyplot as plt
import cf_xarray
from scipy import integrate
import pandas as pd

In [10]:
reprocess = False
if reprocess:
    ds = xr.open_dataset(
        "/lus/scratch/shao/data/NEP36_extremes/3h_old/T.3h.subset.nc",
        chunks="auto"
    )
    cluster_ds = xr.open_dataset(
        "/lus/scratch/shao/data/NEP36_extremes/processed/clim/clim_with_clusters_6.nc"
    )
    ds["cluster"] = cluster_ds.cluster.astype(int)
    ds_clusters = ds.groupby("cluster")
    shallows = ds_clusters[3]
    canyons = ds_clusters[4]
    def save_cluster(ds_cluster, cluster_path, cluster_label):
        encoded = cf_xarray.encode_multi_index_as_compress(ds_cluster, "stacked_y_x")
        encoded.to_netcdf(f"{cluster_path}/{cluster_label}_3h.nc", engine="h5netcdf")
    
    save_cluster(shallows, "./output_data", "shallows")
    save_cluster(canyons, "./output_data", "canyons")

/lus/scratch/shao/local/pyenv/versions/3.10.13/envs/nep36/lib/python3.10/site-packages/xarray/core/duck_array_ops.py:213: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)


In [19]:
ds = dict(
    canyons=xr.open_dataset('output_data/canyons_3h.nc').load(),
    shallows=xr.open_dataset('output_data/shallows_3h.nc').load()
)

In [20]:
resample_frequencies = [
    '3h',
    '6h',
    '12h',
    '1D',
    '5D',
]

ds_resample = {}
hist_resample = {}
for cluster in ("shallows", "canyons"):
    ds_cluster = ds[cluster]
    ds_resample[cluster] = {} 
    
    for resample_frequency in resample_frequencies:
        if resample_frequency = '3h':
            ds_resample[cluster]['3h'] = ds_cluster
        else:
            ds_resample[cluster][resample_frequency] = ds_cluster.resample(time=resample_frequency).mean(dim='time')

In [ ]:
base_frequency = '3h'
nbins = 50

plot_variables = [

    'T'
]

def flatten_and_drop_nan( da ):
    """
    Flatten an xarray DataArray, convert to a 1D numpy, and drop all NaNs
    """
    array = da.to_numpy().flatten()
    array = array[np.isfinite(array)]
    array = array[ array != 0. ]
    return array

var_hist = {}
var_bin_edges = {}

for var in plot_variables:
    var_hist[var] = {}
    hist, bin_edges = np.histogram(
        flatten_and_drop_nan(ds_resample[base_frequency][var]),
        nbins,
        density=True
    )
    var_hist[var]['3h'] = hist.copy()
    var_bin_edges[var] = bin_edges
    for resample_frequency in resample_frequencies:
        print(f'Processing {var} at {resample_frequency}')
        hist, _ = np.histogram(
            flatten_and_drop_nan(ds_resample[resample_frequency][var]),
            bin_edges,
            density=True
        )
        var_hist[var][resample_frequency] = hist.copy()


Processing T at 6h



libgomp: Thread creation failed: Resource temporarily unavailable


In [26]:
downwelling_months = [10, 11, 12, 1, 2, 3]
upwelling_months   = [4, 5, 6, 7, 8, 9]
timeseries_vars = ['O2','OmegaA','T']

base_percentile = 0.1
percentiles = {
    'O2':base_percentile,
    'OmegaA':base_percentile,
    'T':1-base_percentile
}

def filter_by_season_and_values(ds, month_range):       
    ds_out = ds.where(ds['time.month'].isin(month_range),drop=True)
    return ds_out
    
def calculate_threshold(ds, percentile, nbins=1000):
    data = ds.to_numpy().flatten()
    hist, edges = np.histogram(data, bins=nbins, density=True)
    cdf = integrate.cumtrapz(hist,edges[1:])
    return np.interp(percentile, cdf, edges[2:])    

In [41]:
heading = "-"*5
# threshold_df = pd.DataFrame(columns = ["Number of Clusters", "Variable", "Season", "Threshold"])
thresholds = []
var = "T"
percentile = 0.9
for cluster in ["shallows", "canyons"]:
    for frequency in resample_frequencies:
        print(f"Cluster: {cluster}, Frequency: {frequency}")
        downwelling_ds = filter_by_season_and_values(
            ds_resample[cluster][frequency],
            downwelling_months
        )
        upwelling_ds = filter_by_season_and_values(
            ds_resample[cluster][frequency],
            upwelling_months
        )
        
        upwelling_threshold = calculate_threshold(upwelling_ds[var], percentile)
        downwelling_threshold = calculate_threshold(downwelling_ds[var], percentile)

        thresholds.append(
            {
                "Frequency": frequency,
                "Variable": var,
                "Season": "upwelling",
                "Threshold": upwelling_threshold,
                "Cluster": cluster,
            }
        )
        thresholds.append(
            {
                "Frequency": frequency,
                "Variable": var,
                "Season": "downwelling",
                "Threshold": downwelling_threshold,
                "Cluster": cluster,
            }
        )


Cluster: shallows, Frequency: 3h


/tmp/ipykernel_2084426/2448282172.py:19: DeprecationWarning: 'scipy.integrate.cumtrapz' is deprecated in favour of 'scipy.integrate.cumulative_trapezoid' and will be removed in SciPy 1.14.0
  cdf = integrate.cumtrapz(hist,edges[1:])


Cluster: shallows, Frequency: 6h


/tmp/ipykernel_2084426/2448282172.py:19: DeprecationWarning: 'scipy.integrate.cumtrapz' is deprecated in favour of 'scipy.integrate.cumulative_trapezoid' and will be removed in SciPy 1.14.0
  cdf = integrate.cumtrapz(hist,edges[1:])


Cluster: shallows, Frequency: 12h


...

/tmp/ipykernel_2084426/2448282172.py:19: DeprecationWarning: 'scipy.integrate.cumtrapz' is deprecated in favour of 'scipy.integrate.cumulative_trapezoid' and will be removed in SciPy 1.14.0
  cdf = integrate.cumtrapz(hist,edges[1:])


Cluster: canyons, Frequency: 1D


/tmp/ipykernel_2084426/2448282172.py:19: DeprecationWarning: 'scipy.integrate.cumtrapz' is deprecated in favour of 'scipy.integrate.cumulative_trapezoid' and will be removed in SciPy 1.14.0
  cdf = integrate.cumtrapz(hist,edges[1:])


Cluster: canyons, Frequency: 5D


/tmp/ipykernel_2084426/2448282172.py:19: DeprecationWarning: 'scipy.integrate.cumtrapz' is deprecated in favour of 'scipy.integrate.cumulative_trapezoid' and will be removed in SciPy 1.14.0
  cdf = integrate.cumtrapz(hist,edges[1:])


In [36]:


threshold_df = pd.DataFrame(thresholds)

In [40]:
display(threshold_df[(threshold_df["Season"] == "upwelling")])
display(threshold_df[(threshold_df["Season"] == "downwelling")])

,Frequency,Variable,Season,Threshold,Cluster
0,3h,T,upwelling,14.089685,shallows
2,6h,T,upwelling,14.083961,shallows
4,12h,T,upwelling,14.077610,shallows
6,1D,T,upwelling,14.076084,shallows
8,5D,T,upwelling,14.073673,shallows
10,3h,T,upwelling,6.382883,canyons
12,6h,T,upwelling,6.382105,canyons
14,12h,T,upwelling,6.381128,canyons
16,1D,T,upwelling,6.380694,canyons
18,5D,T,upwelling,6.369806,canyons


,Frequency,Variable,Season,Threshold,Cluster
1,3h,T,downwelling,10.681804,shallows
3,6h,T,downwelling,10.681329,shallows
5,12h,T,downwelling,10.680562,shallows
7,1D,T,downwelling,10.679845,shallows
9,5D,T,downwelling,10.580608,shallows
11,3h,T,downwelling,7.050831,canyons
13,6h,T,downwelling,7.050138,canyons
15,12h,T,downwelling,7.049290,canyons
17,1D,T,downwelling,7.048787,canyons
19,5D,T,downwelling,7.047242,canyons
